In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow import keras
import numpy as np

# Generate some sample data for the autoencoder
x_train = np.random.rand(1000, 12)
x_test = np.random.rand(100, 12)

# Define the input layer for the autoencoder
input_layer = keras.Input(shape=(12,))

# Define the encoder layers
encoded = keras.layers.Dense(8, activation='relu')(input_layer)
encoded = keras.layers.Dense(4, activation='relu')(encoded)

# Define the decoder layers
decoded = keras.layers.Dense(8, activation='relu')(encoded)
decoded = keras.layers.Dense(12, activation='sigmoid')(decoded)

# Create the autoencoder model
autoencoder = keras.Model(input_layer, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')

# Train the model
autoencoder.fit(x_train, x_train, epochs=100, batch_size=32, validation_data=(x_test, x_test))

# Use the trained autoencoder to encode the training data
encoded_data = autoencoder.encoder(x_train).numpy()

# Fit a Gaussian Mixture Model with 2 components to the encoded training data
gmm = GaussianMixture(n_components=2)
gmm.fit(encoded_data)

# Use the GMM to predict the probability densities for the encoded test data
encoded_test_data = autoencoder.encoder(x_test).numpy()
scores = gmm.score_samples(encoded_test_data)

# Identify anomalies based on the GMM probability densities
threshold = np.percentile(scores, 5)
anomalies = x_test[scores < threshold]

# Calculate precision, recall, and F1 score
y_true = np.zeros_like(scores)
y_true[scores < threshold] = 1
y_pred = np.zeros_like(scores)
y_pred[scores < threshold] = 1
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 score: {f1:.4f}")
